### Vamos criar um passo a passo para um notebook utilizando a arquitetura Medallion com pandas, dividindo o processo em camadas Bronze, Silver e Gold.

## Bronze: Dados Brutos
Nesta etapa, apenas carregamos os dados brutos(csv) na LandingZone para a camada Bronze e os salvamos em um arquivo Parquet.

### Instalando a Biblioteca pyarrow
Primeiro, vamos instalar a biblioteca pyarrow. Você pode fazer isso usando pip:

In [56]:
!pip install pyarrow
!pip install pandas



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [57]:
import pandas as pd
import numpy as np
import os 

## Etapa 1: Ler arquivo CSV com pandas e salvar em formato Parquet
### Primeiro, precisamos ler o arquivo CSV e salvá-lo no formato Parquet.
### Leitura dos dados brutos da camada Landing Zone

### Ler e salvar arquivo FCX (LZ para Bronze)

In [58]:
import pandas as pd
import os

# Define caminhos para os arquivos
csv_path = r"C:\Fonte Dados\fcx\landingzone\fcx.csv"
parquet_path = r"C:\Fonte Dados\fcx\bronze\fcx.parquet"

# Cria diretórios caso não existam
os.makedirs(os.path.dirname(parquet_path), exist_ok=True)

# Lê o arquivo CSV
df_bronze_fcx = pd.read_csv(csv_path)

# Salva no formato Parquet
df_bronze_fcx.to_parquet(parquet_path, index=False)
print(f"Arquivo salvo em {parquet_path}")



Arquivo salvo em C:\Fonte Dados\fcx\bronze\fcx.parquet


### Ler e salvar arquivo Pessoas (LZ para Bronze)

In [59]:


# Define caminhos para os arquivos
csv_path = r"C:\Fonte Dados\fcx\landingzone\pessoas.csv"
parquet_path = r"C:\Fonte Dados\fcx\bronze\pessoas.parquet"

# Cria diretórios caso não existam
os.makedirs(os.path.dirname(parquet_path), exist_ok=True)

# Lê o arquivo CSV
df_bronze_pessoas = pd.read_csv(csv_path)

# Salva no formato Parquet
df_bronze_pessoas.to_parquet(parquet_path, index=False)
print(f"Arquivo salvo em {parquet_path}")


Arquivo salvo em C:\Fonte Dados\fcx\bronze\pessoas.parquet


### Exibir as primeiras 10 linhas do DataFrame 

In [60]:
print(df_bronze.head(10))

   documento  cod_clifor tipo_doc   valor data_emissão data_vencimento  \
0          1          72    pagar  720472   2023-06-25      2025-09-09   
1          2         257    pagar  403513   2023-11-26      2025-02-19   
2          3          84  receber  471821   2024-05-16      2025-08-17   
3          4         255    pagar  562586   2023-07-09      2023-09-13   
4          5         207  receber  341262   2023-10-07      2024-04-04   
5          6          63    pagar  153930   2023-10-30      2024-01-19   
6          7         364    pagar  259469   2024-06-13      2025-03-19   
7          8         224    pagar   96075   2023-12-18      2025-08-29   
8          9         356    pagar  793772   2023-10-25      2024-10-09   
9         10         198  receber  148451   2023-03-07      2025-11-24   

  data_pagamento condicao_pagamento  
0     2024-07-26            à vista  
1     2024-08-28            à vista  
2     2025-06-22          parcelado  
3     2024-02-11          parcela

### Etapa 2: Desnormalizar dados e criar camada silver
### Define caminhos

In [61]:
silver_path = r"C:\Fonte Dados\fcx\silver" 
bronze_path = r"C:\Fonte Dados\fcx\bronze\fcx.parquet"
bronze_path_pessoas = r"C:\Fonte Dados\fcx\bronze\pessoas.parquet"


In [62]:
# Cria diretórios caso não existam 
os.makedirs(silver_path, exist_ok=True)

In [63]:
# Lê o arquivo Parquet 
df_silver = pd.read_parquet(bronze_path)
df_silver_pessoas = pd.read_parquet(bronze_path_pessoas)

## Ler arquivo Parquet na camada Bronze e Salvar desnormalizado na camadas Silver em tabelas 
### Dimensões
Pessoas (cod_clifor, nomeclifor)
CondicaoPagamento (condicao_pagamento)
TipoDoc (tipo_doc  )
### Fatos
Fato_fcx ( documento, cod_clifor,tipo_doc ,valor, data_emissão,data_vencimento,data_pagamento condicao_pagamento)

In [64]:
# Cria as dimensões 
#Pessoas
df_pessoas = df_silver_pessoas[['cod_clifor', 'nomeclifor']].drop_duplicates() 
#Condicao de Pagamento
df_condicao_pagamento = df_silver[['condicao_pagamento']].drop_duplicates() 
#Tipo de documento
df_tipo_doc = df_silver[['tipo_doc']].drop_duplicates()

In [65]:
df_pessoas

,cod_clifor,nomeclifor
0,1,Larissa da Cruz
1,2,João Teixeira
2,3,Lorenzo Santos
3,4,Maria Cecília Nunes
4,5,Guilherme da Cunha
...,...,...
530,531,Lucas Aragão
531,532,Arthur Santos
532,533,Luara Costela
533,534,Milena Alves


In [66]:
# Filtrando pelo valor específico de cod_clifor (exemplo: 374)
df_selecionado = df_pessoas[df_pessoas['cod_clifor'] == 374]

# Exibindo o resultado
print(df_selecionado)


     cod_clifor      nomeclifor
373         374  Melina Sampaio


In [67]:
df_condicao_pagamento

,condicao_pagamento
0,à vista
2,parcelado


In [68]:
df_tipo_doc

,tipo_doc
0,pagar
2,receber


In [69]:
df_filtrado = df_pessoas[df_pessoas["cod_clifor"] == 202]
df_filtrado

,cod_clifor,nomeclifor
201,202,Fernanda Teixeira


# Cria a dataframe fato 

In [70]:
df_fato_fcx = df_silver[['documento', 'cod_clifor', 'tipo_doc', 'valor', 'data_emissão', 'data_vencimento', 'data_pagamento', 'condicao_pagamento']]

In [71]:
df_fato_fcx

,documento,cod_clifor,tipo_doc,valor,data_emissão,data_vencimento,data_pagamento,condicao_pagamento
0,1,72,pagar,720472,2023-06-25,2025-09-09,2024-07-26,à vista
1,2,257,pagar,403513,2023-11-26,2025-02-19,2024-08-28,à vista
2,3,84,receber,471821,2024-05-16,2025-08-17,2025-06-22,parcelado
3,4,255,pagar,562586,2023-07-09,2023-09-13,2024-02-11,parcelado
4,5,207,receber,341262,2023-10-07,2024-04-04,2024-01-27,à vista
...,...,...,...,...,...,...,...,...
99995,99996,61,receber,47817,2023-06-08,2024-10-01,2025-10-24,à vista
99996,99997,44,receber,445668,2024-10-15,2025-01-17,2025-04-27,parcelado
99997,99998,324,pagar,209812,2023-10-28,2025-03-06,2024-11-13,à vista
99998,99999,16,receber,753606,2023-11-25,2025-06-27,2024-03-18,parcelado


## Salva as dimensões na camada silver

### Criando a dimensão Pessoas

In [72]:
# Exemplo de silver_path armazenado
silver_path = r"C:\Fonte Dados\fcx\silver"
subpasta = 'pessoas'
nome_arquivo = 'pessoas.parquet'

# Construção do caminho para salvar o arquivo
pessoas_path = os.path.join(silver_path, subpasta, nome_arquivo)

# Certifique-se de que a pasta existe, caso contrário, crie-a
os.makedirs(os.path.dirname(pessoas_path), exist_ok=True)

# Salva o dataframe no caminho especificado
df_pessoas.to_parquet(pessoas_path, index=False)

print(f"Arquivo salvo em: {pessoas_path}")

Arquivo salvo em: C:\Fonte Dados\fcx\silver\pessoas\pessoas.parquet


### Criando a dimensão Condicao de Pagamento

In [73]:
# Exemplo de silver_path armazenado
silver_path = r"C:\Fonte Dados\fcx\silver"
subpasta = 'condicao_pagamento'
nome_arquivo = 'condicao_pagamento.parquet'

# Construção do caminho para salvar o arquivo
cp_path = os.path.join(silver_path, subpasta, nome_arquivo)

# Certifique-se de que a pasta existe, caso contrário, crie-a
os.makedirs(os.path.dirname(cp_path), exist_ok=True)

# Salva o dataframe no caminho especificado
df_condicao_pagamento.to_parquet(cp_path, index=False)

print(f"Arquivo salvo em: {cp_path}")

Arquivo salvo em: C:\Fonte Dados\fcx\silver\condicao_pagamento\condicao_pagamento.parquet


### Criando a dimensão Tipo documento

In [74]:
# Exemplo de silver_path armazenado
silver_path = r"C:\Fonte Dados\fcx\silver"
subpasta = 'tipo_doc'
nome_arquivo = 'tipo_doc.parquet'

# Construção do caminho para salvar o arquivo
doc_path = os.path.join(silver_path, subpasta, nome_arquivo)

# Certifique-se de que a pasta existe, caso contrário, crie-a
os.makedirs(os.path.dirname(doc_path), exist_ok=True)

# Salva o dataframe no caminho especificado
df_tipo_doc.to_parquet(doc_path, index=False)

print(f"Arquivo salvo em: {doc_path}")


Arquivo salvo em: C:\Fonte Dados\fcx\silver\tipo_doc\tipo_doc.parquet


### Criar a tabela Fato Transações financeiras particionada Ano/Mês

In [75]:
import os
import pandas as pd

# Exemplo de silver_path armazenado
silver_path = r"C:\Fonte Dados\fcx\silver"
subpasta = 'transacoes'
pasta_completa = os.path.join(silver_path, subpasta)

# Certifique-se de que a coluna 'data_emissão' esteja no formato datetime
df_fato_fcx['data_emissão'] = pd.to_datetime(df_fato_fcx['data_emissão'])

# Adiciona colunas de Ano e Mês
df_fato_fcx['Ano'] = df_fato_fcx['data_emissão'].dt.year
df_fato_fcx['Mes'] = df_fato_fcx['data_emissão'].dt.month

# Salva o arquivo parquet particionado automaticamente
df_fato_fcx.to_parquet(
    pasta_completa,
    index=False,
    partition_cols=['Ano', 'Mes'],
    engine='pyarrow'  # Certifique-se de que o PyArrow está instalado
)

print(f"Arquivo salvo particionado em: {pasta_completa}")


Arquivo salvo particionado em: C:\Fonte Dados\fcx\silver\transacoes


## Evidencia de dados nos arquivos Parquet

In [76]:
# Tabela de Silver - Pessoas 
silver_path = r"C:\Fonte Dados\fcx\silver"
tabela ="pessoas"
pasta_completa = os.path.join(silver_path, tabela)
# Lê o arquivo Parquet 
df_silver = pd.read_parquet(pasta_completa)
df_silver

,cod_clifor,nomeclifor
0,1,Larissa da Cruz
1,2,João Teixeira
2,3,Lorenzo Santos
3,4,Maria Cecília Nunes
4,5,Guilherme da Cunha
...,...,...
530,531,Lucas Aragão
531,532,Arthur Santos
532,533,Luara Costela
533,534,Milena Alves


In [77]:
# Tabela de Silver - Pessoas 
silver_path = r"C:\Fonte Dados\fcx\silver"
tabela ="tipo_doc"
pasta_completa = os.path.join(silver_path, tabela)
# Lê o arquivo Parquet 
df_silver = pd.read_parquet(pasta_completa)
df_silver

,tipo_doc
0,pagar
1,receber


In [78]:
# Tabela de Silver - Pessoas 
silver_path = r"C:\Fonte Dados\fcx\silver"
tabela ="condicao_pagamento"
pasta_completa = os.path.join(silver_path, tabela)
# Lê o arquivo Parquet 
df_silver = pd.read_parquet(pasta_completa)
df_silver

,condicao_pagamento
0,à vista
1,parcelado


In [79]:
# Tabela de Silver - Pessoas 
silver_path = r"C:\Fonte Dados\fcx\silver"
tabela ="transacoes"
pasta_completa = os.path.join(silver_path, tabela)
# Lê o arquivo Parquet 
df_silver = pd.read_parquet(pasta_completa)
df_silver

,documento,cod_clifor,tipo_doc,valor,data_emissão,data_vencimento,data_pagamento,condicao_pagamento,Ano,Mes
0,90,90,pagar,564072,2023-01-29,2024-12-18,2024-08-03,parcelado,2023,1
1,115,245,receber,226841,2023-01-20,2025-10-18,2023-08-01,à vista,2023,1
2,145,402,pagar,337824,2023-01-19,2024-06-22,2024-06-02,parcelado,2023,1
3,184,254,pagar,80057,2023-01-26,2025-12-10,2023-07-24,à vista,2023,1
4,203,375,receber,372949,2023-01-21,2024-11-22,2024-02-09,à vista,2023,1
...,...,...,...,...,...,...,...,...,...,...
99995,99886,155,receber,611973,2025-01-14,2025-05-12,2025-03-09,à vista,2025,1
99996,99902,370,receber,724024,2025-01-02,2025-10-11,2025-06-03,parcelado,2025,1
99997,99913,497,pagar,879713,2025-01-13,2025-09-14,2025-05-25,à vista,2025,1
99998,99976,111,pagar,869196,2025-01-02,2025-09-20,2025-02-10,parcelado,2025,1


## Etapa 3
### Criaçao da Camada Gold (Fatos e dimensões e aplicação SCD1)

In [80]:
import pandas as pd 
import os 
# Define caminhos 
gold_path = r"C:\Fonte Dados\fcx\gold" 
silver_path = r"C:\Fonte Dados\fcx\silver" 
# Cria diretórios caso não existam 

os.makedirs(gold_path, exist_ok=True)

### 1 Ler os arquivos na camada Silver
### 2 Add Surrogate Keys
### 3 Salvar camada Gold


In [81]:
# Lê os dados da camada silver (d) 
tabela = 'pessoas'
pasta_completa = os.path.join(silver_path, tabela)
df_pessoas = pd.read_parquet(os.path.join(pasta_completa)) 


In [82]:
# Cria chaves substitutas (surrogate keys) 
df_pessoas['pessoa_sk'] = range(1, len(df_pessoas) + 1)
df_pessoas

,cod_clifor,nomeclifor,pessoa_sk
0,1,Larissa da Cruz,1
1,2,João Teixeira,2
2,3,Lorenzo Santos,3
3,4,Maria Cecília Nunes,4
4,5,Guilherme da Cunha,5
...,...,...,...
530,531,Lucas Aragão,531
531,532,Arthur Santos,532
532,533,Luara Costela,533
533,534,Milena Alves,534


In [83]:
# Salva as dimensões na camada gold 
subpasta = 'pessoas'
nome_arquivo = 'pessoas.parquet'
pasta_completa = os.path.join(gold_path, tabela, nome_arquivo)
# Certifique-se de que a pasta existe, caso contrário, crie-a
os.makedirs(os.path.dirname(pasta_completa), exist_ok=True)
#Salva Arquivo Parquet na Gold
df_pessoas.to_parquet(pasta_completa, index=False)
print(f"Arquivo salvo em: {pasta_completa}")





Arquivo salvo em: C:\Fonte Dados\fcx\gold\pessoas\pessoas.parquet


### Dimensão Comdicao de Pagamento

In [84]:
# Lê os dados da camada silver (d) 
subpasta = 'condicao_pagamento'
pasta_completa = os.path.join(silver_path, subpasta)
df_condicao_pagamento = pd.read_parquet(os.path.join(pasta_completa)) 
# Cria chaves substitutas (surrogate keys) 
df_condicao_pagamento['cp_sk'] = range(1, len(df_condicao_pagamento) + 1)
# Salva as dimensões na camada gold 
nome_arquivo = 'condicao_pagamento.parquet'
pasta_completa = os.path.join(gold_path, subpasta, nome_arquivo)
# Certifique-se de que a pasta existe, caso contrário, crie-a
os.makedirs(os.path.dirname(pasta_completa), exist_ok=True)
#Salva Arquivo Parquet na Gold
df_condicao_pagamento.to_parquet(pasta_completa, index=False)
print(f"Arquivo salvo em: {pasta_completa}")



Arquivo salvo em: C:\Fonte Dados\fcx\gold\condicao_pagamento\condicao_pagamento.parquet


### Dimensão Tipo Documento

In [85]:
# Lê os dados da camada silver (d) 
subpasta = 'tipo_doc'
pasta_completa = os.path.join(silver_path, subpasta)
df_tipo_doc = pd.read_parquet(os.path.join(pasta_completa)) 
# Cria chaves substitutas (surrogate keys) 
df_tipo_doc['td_sk'] = range(1, len(df_tipo_doc) + 1)
# Salva as dimensões na camada gold 
nome_arquivo = 'condicao_pagamento.parquet'
pasta_completa = os.path.join(gold_path, subpasta, nome_arquivo)
# Certifique-se de que a pasta existe, caso contrário, crie-a
os.makedirs(os.path.dirname(pasta_completa), exist_ok=True)
#Salva Arquivo Parquet na Gold
df_tipo_doc.to_parquet(pasta_completa, index=False)
print(f"Arquivo salvo em: {pasta_completa}")

Arquivo salvo em: C:\Fonte Dados\fcx\gold\tipo_doc\condicao_pagamento.parquet


## Construção da Tabela Fato

In [86]:
# Lê os dados da camada silver (d) 
subpasta = 'transacoes'
pasta_completa = os.path.join(silver_path, subpasta)
df_fato_fcx = pd.read_parquet(os.path.join(pasta_completa)) 
df_fato_fcx

,documento,cod_clifor,tipo_doc,valor,data_emissão,data_vencimento,data_pagamento,condicao_pagamento,Ano,Mes
0,90,90,pagar,564072,2023-01-29,2024-12-18,2024-08-03,parcelado,2023,1
1,115,245,receber,226841,2023-01-20,2025-10-18,2023-08-01,à vista,2023,1
2,145,402,pagar,337824,2023-01-19,2024-06-22,2024-06-02,parcelado,2023,1
3,184,254,pagar,80057,2023-01-26,2025-12-10,2023-07-24,à vista,2023,1
4,203,375,receber,372949,2023-01-21,2024-11-22,2024-02-09,à vista,2023,1
...,...,...,...,...,...,...,...,...,...,...
99995,99886,155,receber,611973,2025-01-14,2025-05-12,2025-03-09,à vista,2025,1
99996,99902,370,receber,724024,2025-01-02,2025-10-11,2025-06-03,parcelado,2025,1
99997,99913,497,pagar,879713,2025-01-13,2025-09-14,2025-05-25,à vista,2025,1
99998,99976,111,pagar,869196,2025-01-02,2025-09-20,2025-02-10,parcelado,2025,1


In [87]:
# Junta a tabela fato com as chaves substitutas
df_fato_fcx = (
    df_fato_fcx
    .merge(df_pessoas, on='cod_clifor')
    .merge(df_condicao_pagamento, on='condicao_pagamento')
    .merge(df_tipo_doc, on='tipo_doc')
)

# Seleciona as colunas da tabela fato com as chaves substitutas
df_fato_fcx_gold = df_fato_fcx[['documento', 'pessoa_sk', 'td_sk', 'cp_sk', 'valor', 'data_emissão', 'data_vencimento', 'data_pagamento']]

# Exibe o DataFrame final
df_fato_fcx_gold


,documento,pessoa_sk,td_sk,cp_sk,valor,data_emissão,data_vencimento,data_pagamento
0,90,90,1,2,564072,2023-01-29,2024-12-18,2024-08-03
1,115,245,2,1,226841,2023-01-20,2025-10-18,2023-08-01
2,145,402,1,2,337824,2023-01-19,2024-06-22,2024-06-02
3,184,254,1,1,80057,2023-01-26,2025-12-10,2023-07-24
4,203,375,2,1,372949,2023-01-21,2024-11-22,2024-02-09
...,...,...,...,...,...,...,...,...
99995,99886,155,2,1,611973,2025-01-14,2025-05-12,2025-03-09
99996,99902,370,2,2,724024,2025-01-02,2025-10-11,2025-06-03
99997,99913,497,1,1,879713,2025-01-13,2025-09-14,2025-05-25
99998,99976,111,1,2,869196,2025-01-02,2025-09-20,2025-02-10


# Salva a tabela fato na camada gold 


In [88]:
import os
import pandas as pd

# Exemplo de silver_path armazenado
gold_path = r"C:\Fonte Dados\fcx\gold"
subpasta = 'transacoes'
pasta_completa = os.path.join(gold_path, subpasta)

# Certifique-se de que a coluna 'data_emissão' esteja no formato datetime
df_fato_fcx_gold['data_emissão'] = pd.to_datetime(df_fato_fcx_gold['data_emissão'])

# Adiciona colunas de Ano e Mês
df_fato_fcx_gold['Ano'] = df_fato_fcx_gold['data_emissão'].dt.year
df_fato_fcx_gold['Mes'] = df_fato_fcx_gold['data_emissão'].dt.month

# Salva o arquivo parquet particionado automaticamente
df_fato_fcx_gold.to_parquet(
    pasta_completa,
    index=False,
    partition_cols=['Ano', 'Mes'],
    engine='pyarrow'  # Certifique-se de que o PyArrow está instalado
)

print(f"Arquivo salvo particionado em: {pasta_completa}")


Arquivo salvo particionado em: C:\Fonte Dados\fcx\gold\transacoes


C:\Users\andre\AppData\Local\Temp\ipykernel_20456\411763421.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fato_fcx_gold['data_emissão'] = pd.to_datetime(df_fato_fcx_gold['data_emissão'])
C:\Users\andre\AppData\Local\Temp\ipykernel_20456\411763421.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fato_fcx_gold['Ano'] = df_fato_fcx_gold['data_emissão'].dt.year
C:\Users\andre\AppData\Local\Temp\ipykernel_20456\411763421.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

## Consulta Particao

In [89]:
import pandas as pd
import os

# Caminho base da pasta gold
gold_path = r"C:\Fonte Dados\fcx\gold"
subpasta = 'transacoes'

# Caminho da partição específica (Ano=2023/Mes=5)
caminho_particao = os.path.join(gold_path, subpasta, "Ano=2023", "Mes=5")

# Lê os arquivos Parquet na partição
df_fato_gold_2023_5 = pd.read_parquet(caminho_particao)

# Exibe o DataFrame resultante
print(df_fato_gold_2023_5)


      documento  pessoa_sk  td_sk  cp_sk   valor data_emissão data_vencimento  \
0            22        366      2      2  452611   2023-05-24      2023-07-18   
1            32        356      2      1  283944   2023-05-05      2024-01-23   
2            77        408      1      2  833074   2023-05-31      2025-11-04   
3           105         50      1      1  780365   2023-05-02      2024-12-27   
4           142        354      2      2  422324   2023-05-28      2024-03-04   
...         ...        ...    ...    ...     ...          ...             ...   
4185      99919        372      1      2  282302   2023-05-29      2025-06-10   
4186      99924        507      1      2  419125   2023-05-25      2025-05-17   
4187      99961        454      1      1   65300   2023-05-17      2024-09-15   
4188      99971         53      2      1  208392   2023-05-08      2025-12-06   
4189      99988        327      2      1  387179   2023-05-24      2023-09-04   

     data_pagamento  
0    

In [90]:
print(df_fato_fcx.columns)
print(df_fato_fcx_gold.columns)
print(df_pessoas.columns)

Index(['documento', 'cod_clifor', 'tipo_doc', 'valor', 'data_emissão',
       'data_vencimento', 'data_pagamento', 'condicao_pagamento', 'Ano', 'Mes',
       'nomeclifor', 'pessoa_sk', 'cp_sk', 'td_sk'],
      dtype='object')
Index(['documento', 'pessoa_sk', 'td_sk', 'cp_sk', 'valor', 'data_emissão',
       'data_vencimento', 'data_pagamento', 'Ano', 'Mes'],
      dtype='object')
Index(['cod_clifor', 'nomeclifor', 'pessoa_sk'], dtype='object')


### 1. Verifique a unicidade das chaves antes do merge
Use o código abaixo para verificar se as chaves usadas no merge são únicas em cada DataFrame:


In [91]:

# Verifica duplicatas nas chaves
print("Duplicatas em df_fato_fcx:", not df_fato_fcx['cod_clifor'].is_unique)
print("Duplicatas em df_pessoas:", not df_pessoas['cod_clifor'].is_unique)
print("Duplicatas em df_condicao_pagamento:", not df_condicao_pagamento['condicao_pagamento'].is_unique)
print("Duplicatas em df_tipo_doc:", not df_tipo_doc['tipo_doc'].is_unique)


Duplicatas em df_fato_fcx: True
Duplicatas em df_pessoas: False
Duplicatas em df_condicao_pagamento: False
Duplicatas em df_tipo_doc: False


In [95]:
df_pessoas.shape

(535, 3)

In [94]:
df_pessoas.describe

<bound method NDFrame.describe of      cod_clifor           nomeclifor  pessoa_sk
0             1      Larissa da Cruz          1
1             2        João Teixeira          2
2             3       Lorenzo Santos          3
3             4  Maria Cecília Nunes          4
4             5   Guilherme da Cunha          5
..          ...                  ...        ...
530         531         Lucas Aragão        531
531         532        Arthur Santos        532
532         533        Luara Costela        533
533         534         Milena Alves        534
534         535     Sra. Liz da Cruz        535

[535 rows x 3 columns]>

In [96]:
df_pessoas.sample

<bound method NDFrame.sample of      cod_clifor           nomeclifor  pessoa_sk
0             1      Larissa da Cruz          1
1             2        João Teixeira          2
2             3       Lorenzo Santos          3
3             4  Maria Cecília Nunes          4
4             5   Guilherme da Cunha          5
..          ...                  ...        ...
530         531         Lucas Aragão        531
531         532        Arthur Santos        532
532         533        Luara Costela        533
533         534         Milena Alves        534
534         535     Sra. Liz da Cruz        535

[535 rows x 3 columns]>

In [97]:
df_fato_fcx_gold.info

<bound method DataFrame.info of        documento  pessoa_sk  td_sk  cp_sk   valor data_emissão  \
0             90         90      1      2  564072   2023-01-29   
1            115        245      2      1  226841   2023-01-20   
2            145        402      1      2  337824   2023-01-19   
3            184        254      1      1   80057   2023-01-26   
4            203        375      2      1  372949   2023-01-21   
...          ...        ...    ...    ...     ...          ...   
99995      99886        155      2      1  611973   2025-01-14   
99996      99902        370      2      2  724024   2025-01-02   
99997      99913        497      1      1  879713   2025-01-13   
99998      99976        111      1      2  869196   2025-01-02   
99999      99979        494      1      2  697479   2025-01-02   

      data_vencimento data_pagamento   Ano  Mes  
0          2024-12-18     2024-08-03  2023    1  
1          2025-10-18     2023-08-01  2023    1  
2          2024-06-22    